In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
# reads mnist data.
# mnist has training set (mnist.train), test set (mnist.test),
# and validation set (mnist.validation)
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz


In [ ]:
### NOTES REGARDING DATA SET
## Training Set (55,000 dp)
# mnist.train.images -> the "x"s.
# - each image is 28x28 pixels.
# - we will flatten this array to make a 784-d vector space.
# - note: flattening data throws away info.
#   - for future, don't do this! learn better methods!
# - this images is a tensor with a shape of [55000, 784].
#   - first-d is index into the list of images.
#   - second-d is index for each pixel.
#     - each entry in the tensor is a pixel intensity between 0-1
# mnist.train.labels -> the "y"s.
# - each image has a corresponding label, a # between 0 and 9.
# - we want the "one-hot vector"
#   - one-hot-vector: a vector which is 0 in most dimensions, and
#      1 in a single dimension.
#   - 3 would be [0,0,0,1,0,0,0,0,0,0]
# - thus, mist.train.labels is a [55000, 10] array of floats.
## Test Set (10,000 dp)
## Validation Set (5,000 dp)

In [ ]:
### SOFTMAX REGRESSIONS
## Objective: look at an image -> get possibility for it being each digit
## Usage:
# if you want to assign ***probabilities*** to an object being
#   one of several different things, softmax is the thing to do.
## Steps:
# 1. add up the evidence of our input being in certain classes.
#   - to do this, we do a weighted sum of the pixel intensities.
#   - the weight is negative if that pixel having a high intensity
#     is evidence against the image being in that weight,
#     and positive if it is evidence in favor.
# 2.  convert evidence into probabilities.